I don't have very much experience in **R** aside from a ~8 hour bootcamp Udemy course I took in Jan 2022. I coded (most of) my entire thesis work in Python mostly through scripts in the command line, and did most of my portfolio and online data science/ML courses using Python with Jupyter notebooks - so Python is much more comfortable to me at this time. I however made it a priority to do the *Analysis* portion of this exercise in **R** as I am aware from the job posting and first interview that **R** is more standard for programming with your company. Given the opportunity to work with you all, I would be very excited to learn more **R**/Tidyverse!

While I appreciate Jupyter Notebooks are NOT the standard for **R** coding, I think this format will allow for better commenting of my work.

This portion of the project is done in a Jupyter Notebook, this code is in **R** and was done in Google Colab.

In [1]:
install.packages("tidyverse")
install.packages("lubridate")
install.packages("openxlsx", dependencies = TRUE)

library(tidyverse)
library(lubridate)
library(openxlsx)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘textshaping’, ‘ragg’


Warning message in install.packages("tidyverse"):
“installation of package ‘textshaping’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘ragg’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘tidyverse’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::l

First I will load the CSV files into tables.

In [2]:
aTC <- read_csv('DataAnalyst_Ecom_data_addsToCart.csv') #addsToCart table
sC <- read_csv('DataAnalyst_Ecom_data_sessionCounts.csv') #sessionCounts table

Rows: 12 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (3): dim_year, dim_month, addsToCart

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 7734 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): dim_browser, dim_deviceCategory, dim_date
dbl (3): sessions, transactions, QTY

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
#check the first few rows of each
head(aTC)

dim_year,dim_month,addsToCart
<dbl>,<dbl>,<dbl>
2012,7,191504
2012,8,217666
2012,9,123726
2012,10,139803
2012,11,186572
2012,12,168972


In [4]:
head(sC)

dim_browser,dim_deviceCategory,dim_date,sessions,transactions,QTY
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
Safari,tablet,7/1/12,2928,127,221
Internet Explorer,desktop,7/1/12,1106,28,0
Chrome,tablet,7/1/12,474,3,13
Amazon Silk,tablet,7/1/12,235,4,5
Internet Explorer,mobile,7/1/12,178,6,11
Internet Explorer,tablet,7/1/12,120,7,0


The **sessionCounts* table's *dim_date* variable is in mo/day/year format, so we'll want to break that up.

As I'm new to doing this sort of thing in **R** I Googled this, and found the following link: https://community.rstudio.com/t/converting-dates-year-month-day-to-3-separate-columns-year-month-day/8585/2

"A" process is using the *lubridate* library (deals with date-times) and then through *dplyr*'s *mutate* and with syntax:

```
x %>%
  dplyr::mutate(year = lubridate::year(date),
                month = lubridate::month(date),
                day = lubridate::day(date))
```



In [5]:
#test code here to troubleshoot the date conversion
p <- mdy(sC$dim_date[1]) #mdy takes my current mo/day/year format and converts it

year(p)
month(p)
day(p)

[1] 2012

[1] 7

[1] 1

In [6]:
#This command will split my date into month, day, and year, and create new columns for those

sC = sC %>%
  dplyr::mutate(dim_year = lubridate::year(mdy(dim_date)),
                dim_month = lubridate::month(mdy(dim_date)),
                dim_day = lubridate::day(mdy(dim_date)))

In [7]:
head(sC)

dim_browser,dim_deviceCategory,dim_date,sessions,transactions,QTY,dim_year,dim_month,dim_day
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
Safari,tablet,7/1/12,2928,127,221,2012,7,1
Internet Explorer,desktop,7/1/12,1106,28,0,2012,7,1
Chrome,tablet,7/1/12,474,3,13,2012,7,1
Amazon Silk,tablet,7/1/12,235,4,5,2012,7,1
Internet Explorer,mobile,7/1/12,178,6,11,2012,7,1
Internet Explorer,tablet,7/1/12,120,7,0,2012,7,1


As *dim_date* is now redundant and *dim_day* doesn't show up in the **addsToCart** table, I will remove them from the **sessionCounts** table.

In [8]:
sC <- select(sC, -c(dim_date,dim_day))

In [9]:
head(sC)

dim_browser,dim_deviceCategory,sessions,transactions,QTY,dim_year,dim_month
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Safari,tablet,2928,127,221,2012,7
Internet Explorer,desktop,1106,28,0,2012,7
Chrome,tablet,474,3,13,2012,7
Amazon Silk,tablet,235,4,5,2012,7
Internet Explorer,mobile,178,6,11,2012,7
Internet Explorer,tablet,120,7,0,2012,7


Now we need to aggregate the data in the **sessionCounts** table, ignoring *dim_browser* and *dim_deviceCategory* columns. These however may be useful for other exploratory data later - so it might make sense to aggregate the other metrics into the **addsToCart** table.

This also is at the edge of my knowledge in **R**, so I'm going to reference a Python Pandas to Tidyverse site: https://thatdatatho.com/from-tidyverse-to-pandas-and-back-an-introduction-to-data-wrangling-with-pyhton-and-r/

In all honesty, while I have exposure to the Python alternative in Pandas for such a method, I probably would have done most of this manually with *For Loops* had I not struck upon this example. As the approach here is far more fluid, I will not attempt to loop this and thank those from that link above.

In [10]:
temp_sC <- sC %>%
  dplyr::select(-c(dim_browser,dim_deviceCategory)) %>% #ignore dim_browser and dim_deviceCategory
  dplyr::group_by(dim_year,dim_month) %>%
  #dplyr::summarise(sessions_month = mean(sessions, na.rm = TRUE)) %>%
  dplyr::summarise(sessions_month = mean(sessions, na.rm = TRUE),
    transactions_month = mean(transactions, na.rm = TRUE),
    QTY_month = mean(QTY, na.rm = TRUE)) %>%
  #dplyr::summarise(transactions_month = mean(transactions, na.rm = TRUE)) %>%
  #dplyr::summarise(QTY_month = mean(QTY, na.rm = TRUE)) %>%
  dplyr::ungroup() %>%
  dplyr::arrange(dim_year,dim_month)

`summarise()` has grouped output by 'dim_year'. You can override using the
`.groups` argument.


In [11]:
head(temp_sC)

dim_year,dim_month,sessions_month,transactions_month,QTY_month
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2012,7,1325.153,31.31207,54.83448
2012,8,1373.110,32.18531,57.84307
2012,9,1179.098,27.86121,50.58007
2012,10,1097.528,24.15398,45.05245
2012,11,1062.967,25.87833,46.88667
2012,12,1257.379,31.73408,55.33758


In [12]:
#add new column ECR

temp_sC <- temp_sC %>%
  mutate(ECR_month=transactions_month/sessions_month)

Now add Aggregated data to the original **addsToCart** table. We'll do an inner join on month+year.

In [13]:
agg_data <- aTC %>% inner_join(temp_sC)

Joining with `by = join_by(dim_year, dim_month)`


In [14]:
head(agg_data)

dim_year,dim_month,addsToCart,sessions_month,transactions_month,QTY_month,ECR_month
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2012,7,191504,1325.153,31.31207,54.83448,0.02362901
2012,8,217666,1373.110,32.18531,57.84307,0.02343971
2012,9,123726,1179.098,27.86121,50.58007,0.02362926
2012,10,139803,1097.528,24.15398,45.05245,0.02200762
2012,11,186572,1062.967,25.87833,46.88667,0.02434539
2012,12,168972,1257.379,31.73408,55.33758,0.02523827


Now for the *Month over Month comparison (for the most recent two months in the data)* calculations.

I am new to industry, this is not a calculation I was familiar with, so I googled some examples: #https://www.infoworld.com/article/3404276/how-to-calculate-month-over-month-changes-in-r.html

So,

${\rm Month \ over \ Month} = \frac{\rm Current \ Month - Last \ Month}{\rm Last \ Month}$

and by extension *over the last two months* might mean,

${\rm Month \ over \ Month \ (two \ months)} = \frac{\rm Current \ Month - Month \ Before \ Last}{\rm Month \ Before \ Last}$

As I'm not 100% sure of the later, I'm emailed that question, and was told the *two month* equation was an appropriate interpretation of that question.

Note: *lag* function shifts the time step back *n* iterations, assuming everything is sorted


In [15]:
#manually create the MoM values

#I did try this using dplyr methods, however ran into an issue where I got NA values, presumably due to rounding - 5-10 minutes of Googling didn't solve the problem
MoM_dim_year = agg_data$dim_year
MoM_dim_month = agg_data$dim_month
MoM_addsToCart = round((agg_data$addsToCart - lag(agg_data$addsToCart)) / lag(agg_data$addsToCart) * 100,1)
MoM_sessions = round((agg_data$sessions_month - lag(agg_data$sessions_month)) / lag(agg_data$sessions_month) * 100,1)
MoM_transactions = round((agg_data$transactions_month - lag(agg_data$transactions_month)) / lag(agg_data$transactions_month) * 100,1)
MoM_QTY = round((agg_data$QTY_month - lag(agg_data$QTY_month)) / lag(agg_data$QTY_month) * 100,1)
MoM_ECR = round((agg_data$ECR_month - lag(agg_data$ECR_month)) / lag(agg_data$ECR_month) * 100,1)

In [16]:
#put MoM arrays into a table
MoM <- as_tibble(data.frame(MoM_dim_year,MoM_dim_month,
  MoM_addsToCart,MoM_sessions,MoM_transactions,MoM_QTY,MoM_ECR))

MoM <- MoM[-1,] #get rid of NA row created before there was data to iterate over

In [17]:
#check calculation
head(MoM)

MoM_dim_year,MoM_dim_month,MoM_addsToCart,MoM_sessions,MoM_transactions,MoM_QTY,MoM_ECR
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2012,8,13.7,3.6,2.8,5.5,-0.8
2012,9,-43.2,-14.1,-13.4,-12.6,0.8
2012,10,13.0,-6.9,-13.3,-10.9,-6.9
2012,11,33.5,-3.1,7.1,4.1,10.6
2012,12,-9.4,18.3,22.6,18.0,3.7
2013,1,-12.6,17.0,11.0,14.7,-5.1


In [18]:
# MoM for most recent two months

#manually create the MoM values, since the above doesn't work
MoM2_dim_year = agg_data$dim_year
MoM2_dim_month = agg_data$dim_month
MoM2_addsToCart = round((agg_data$addsToCart - lag(agg_data$addsToCart,2)) / lag(agg_data$addsToCart,2) * 100,1)
MoM2_sessions = round((agg_data$sessions_month - lag(agg_data$sessions_month,2)) / lag(agg_data$sessions_month,2) * 100,1)
MoM2_transactions = round((agg_data$transactions_month - lag(agg_data$transactions_month,2)) / lag(agg_data$transactions_month,2) * 100,1)
MoM2_QTY = round((agg_data$QTY_month - lag(agg_data$QTY_month,2)) / lag(agg_data$QTY_month,2) * 100,1)
MoM2_ECR = round((agg_data$ECR_month - lag(agg_data$ECR_month,2)) / lag(agg_data$ECR_month,2) * 100,1)
MoM2 <- as_tibble(data.frame(MoM2_dim_year,MoM2_dim_month,
  MoM2_addsToCart,MoM2_sessions,MoM2_transactions,MoM2_QTY,MoM2_ECR))

MoM2 <- MoM2[-1,] #get rid of NA rows created before there was data to iterate over
MoM2 <- MoM2[-1,] #get rid of NA rows created before there was data to iterate over
head(MoM2)

MoM2_dim_year,MoM2_dim_month,MoM2_addsToCart,MoM2_sessions,MoM2_transactions,MoM2_QTY,MoM2_ECR
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2012,9,-35.4,-11.0,-11.0,-7.8,0.0
2012,10,-35.8,-20.1,-25.0,-22.1,-6.1
2012,11,50.8,-9.8,-7.1,-7.3,3.0
2012,12,20.9,14.6,31.4,22.8,14.7
2013,1,-20.9,38.3,36.1,35.4,-1.6
2013,2,-19.6,-19.1,-17.5,-9.7,2.0


For Excel File:

– The first sheet should contain a Month * Device aggregation of the data with the following metrics: Sessions, Transactions, QTY, and ECR (= Transactions / Sessions)

– The second sheet should contain a Month over Month comparison (for the most recent two months in the data) for all available metrics (including Adds to Cart), showing: the most recent month’s value, the prior month’s value, and both the absolute and relative differences between them

In [19]:
#https://www.statology.org/r-export-to-excel-multiple-sheets/

#this is the Excel file output
#ForTheBoss <- list('Month*DeviceAgg' = agg_data, 'MonthOverMonthIn%' = MoM, 'MonthOverMonthIn(2Months)%' = MoM2)
ForTheBoss <- list('Month*DeviceAgg' = agg_data, 'MonthOverMonthIn(2Months)%' = MoM2)


#export each data frame to separate sheets in same Excel file
openxlsx::write.xlsx(ForTheBoss, file = 'ForTheBoss.xlsx')


This chunk of the project took about 3.5-4 hours.

For the Figures I am going to export this data into a few CSVs, then I'll read them into Python in a second .ipynb file. I exceeded my knowledge of **R**, noting that I knew very little going into this portion of the project, and learned a lot in the process.


In [20]:
write.table(agg_data, file = "DataAnalyst_Ecom_agg_data.csv")
write.table(MoM2, file = "DataAnalyst_Ecom_MOM_data.csv")
write.table(sC, file = "DataAnalyst_Ecom_session_with_yr_mo_data.csv")